# Assignment 1: Inspecting the Dataset

In this first assignment, we will get to know the dataset that we will be using throughout. You can find the assignment tasks at the bottom of this document.

Our dataset consists of short texts (article abstracts) from the [PubMed](https://www.ncbi.nlm.nih.gov/pubmed/) database of scientific publications in the Life Science domain. As the full dataset consists of millions of documents, we are using just a small subset, namely all publications that contain the word "influenza" in their title or abstract. You can download that dataset in the form of four files (`influenza_Summaries.pkl.bz2`, etc.) from Canvas. Save these four files in a directory called `data` (do not unpack them, but save them as .bz2 files!), which should be a sub-directory of the one that contains this notebook file (or adjust the file path in the code).

## Importing some Libraries

In [ ]:
from collections import namedtuple, defaultdict, Counter
import pickle, bz2

## Loading the Dataset

In [ ]:
Summaries_file = 'data/influenza_Summaries.pkl.bz2'
Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )

To make it easier to access the data, we convert here paper entries into [named tuples](http://docs.python.org/3/library/collections.html#collections.namedtuple). This will allow us to refer to fields by keyword (like `var.year`), rather than index (like `var[2]`).

In [ ]:
paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )

for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [ ]:
Summaries[28953867]

In [ ]:
Summaries[28953867].title

## Dataset Statistics

The two lines below will install [matplotlib](http://matplotlib.org/) and [NumPy](http://www.numpy.org/), if they are not installed already (if that doesn't work, you might have to download and install them from [here](http://matplotlib.org/downloads.html) and [here](http://sourceforge.net/projects/numpy/files/NumPy/1.10.1/)). We will need these libraries for drawing diagrams and analyzing the data below.

In [ ]:
! pip install matplotlib
! pip install numpy

In [ ]:
import matplotlib.pyplot as plt

# show plots inline within the notebook
%matplotlib inline
# set plots' resolution
plt.rcParams['savefig.dpi'] = 100

### Papers per Year

First, we will try to find out how many papers in the dataset were published per year. We are using the [Counter](https://docs.python.org/3/library/collections.html#collections.Counter) class for this.

In [ ]:
paper_years = [ p.year for p in Summaries.values() ]
papers_per_year = sorted( Counter(paper_years).items() )

print('Number of papers in the dataset per year for the past decade:')
print(papers_per_year[-10:])

Filtering results, to obain only papers since 1940:

In [ ]:
papers_per_year_since_1940 = [ (y,count) for (y,count) in papers_per_year if y >= 1940 ]
years_since_1940 = [ y for (y,count) in papers_per_year_since_1940 ]
nr_papers_since_1940 = [ count for (y,count) in papers_per_year_since_1940 ]

print('Number of papers in the dataset published since 1940:')
print(sum(nr_papers_since_1940))

Creating a bar plot to visualize the results (using [matplotlib.pyplot.bar](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.bar)):

In [ ]:
plt.bar(x=years_since_1940, height=nr_papers_since_1940, width=1.0)
plt.xlim(1940, 2019)
plt.xlabel('year')
plt.ylabel('number of papers');

Alternatively, we can get the same result by plotting it as a histogram with [matplotlib.pyplot.hist](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.hist):

In [ ]:
plt.hist( x=[p.year for p in Summaries.values()], bins=range(1940,2020) );
plt.xlim(1940, 2019)
plt.xlabel('year')
plt.ylabel('number of papers');

### Papers per Author

Here, we will obtain the distribution characterizing the number of papers published by an author.

In [ ]:
# flattening the list of lists of authors:
flat_author_list = [ auth for paper in Summaries.values() for auth in paper.authors ]

nr_papers_by_author = Counter( flat_author_list )

In [ ]:
print('Number of authors in the dataset with distinct names:')
print(len(nr_papers_by_author))

In [ ]:
print('Top 50 authors with greatest number of papers:')
print(sorted(nr_papers_by_author.items(), key=lambda i:i[1], reverse=True)[:50])

Then we can create a histogram to visualize the results:

In [ ]:
plt.hist( x=list(nr_papers_by_author.values()), bins=range(51), log=True )
plt.xlabel('number of papers authored')
plt.ylabel('number of authors');

### Authors per Paper

Then we can make a similar analysis for the number of authors per paper:

In [ ]:
plt.hist( x=[ len(p.authors) for p in Summaries.values() ], bins=range(20), align='left', normed=True )
plt.xlabel('number of authors in one paper')
plt.ylabel('fraction of papers')
plt.xlim(-0.5, 15.5);

### Words in Titles

And then we can have a first look into the words we find in titles (applying a very naive word splitting method for now):

In [ ]:
words = [ word.lower() for paper in Summaries.values() for word in paper.title.split(' ') ]
word_counts = Counter(words)

print('Number of distinct words in the paper titles:')
print(len(word_counts))

----------

# Tasks

**Your name:** David Rocker

### Task 1

Create a Python dictionary object that returns sets of author names for a given year. Name this dictionary `authors_at_year`. (You can use a [*defaultdict*](https://docs.python.org/3/library/collections.html#collections.defaultdict) with a default value of [*set*](https://docs.python.org/3/library/stdtypes.html#set-types-set-frozenset).) Demonstrate the working of this dictionary by showing the author set for the year 1900.

In [ ]:
authors_at_year = defaultdict(set)
for p in Summaries.values():
    for author in p.authors:
        authors_at_year[p.year].add(author)
print(authors_at_year[1900])

### Task 2

Based on the dictionary `authors_at_year` from exercise 1 above, create a plot for the years from 1940 until now that shows how many authors published at least one paper. (You can retrieve the number of unique items in a set `s` with `len(s)`.)

In [ ]:
number_authors_per_year = defaultdict(int)
for i in authors_at_year:
    if i >= 1940:
        number_authors_per_year[i] = (len(authors_at_year[i]))

for x in number_authors_per_year:
    plt.plot(x,number_authors_per_year[x],'o')
plt.xlim(1940,2020)
plt.xlabel('Year')
plt.ylabel('Number Authors')



### Task 3

Print out the top 10 most often occurring words in the papers' titles together with their frequency (in descending order with respect to the frequency). You can make use of the data structures created above.

In [ ]:
print(word_counts.most_common(10))

### Task 4

Calculate and plot (e.g. using [plt.plot](http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.plot)) a graph of the frequency of the 100 most frequent words in titles of papers, from most frequent to least frequent. (For readability, don't show the actual words.) You can make use of the data structures created above.

In [ ]:
for x,y in word_counts.most_common(100):
    plt.plot(x,y, 'o')
plt.ylabel('Frequency')
plt.xlabel('Words')
plt.gca().axes.get_xaxis().set_ticklabels([])

### Task 5

What do you notice when looking at the top 10 most frequent words? Explain what types of words we find in this top-10 and why.

**Answer:** The worlds are mostly filler words such as: the, of, in, and, etc. This type of word is most common in the english language, when doing NLP normally you will scrub the data to remove filler words and only focus on the words that have meaning to the paper. 

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.